In [ ]:
# imports
from typing import Tuple, List
import time
from os import path
import requests

import urllib
import zipfile
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Exercice 1 : 

In [ ]:
url = u'https://archive.ics.uci.edu/ml/machine-learning-databases/00235/'
filename = 'household_power_consumption'
zipfilename = filename + '.zip'
location = url + zipfilename
if not path.isfile(zipfilename):
    urllib.request.urlretrieve(location, zipfilename)
zipfile.ZipFile(zipfilename).extractall()
na_values = ['?', '']
fields = ['Date', 'Time', 'Global_active_power']
df = pd.read_csv(filename + '.txt', sep=';', nrows=200000,
                 na_values=na_values, usecols=fields)

## Question 1 : 

In [ ]:
# count missing values
df.isnull().sum()

In [ ]:
# delete rows with missing values
df = df.dropna()

In [ ]:
# sanity check
df.isnull().sum()

# Question 2 :

In [ ]:
df['DateTime'] = df['Date'] +' '+ df['Time']

In [ ]:
# datetime to timetime 
df['DateTime'] = pd.to_datetime(df['DateTime'])

In [ ]:
# set index to DateTime
df = df.set_index('DateTime')


In [ ]:
# Time and Date are now useless 
df.drop(['Date', 'Time'], axis=1, inplace=True)


In [ ]:
df.head()

# question 3 : 

In [ ]:
# average power consumption per day 
df_daily = df.resample('D').mean()

# plot global_active_power between january 1 2007 and april 30 2007 
df_daily.loc['2007-01-01':'2007-04-30'].plot(y='Global_active_power')



## Question 4 :

In [ ]:

# load https://perso.telecom-paristech.fr/sabourin/mdi720/TG_STAID011249.txt
url = 'https://perso.telecom-paristech.fr/sabourin/mdi720/TG_STAID011249.txt'
response = requests.get(url)
temp_txt = response.text

# some custom code to clean text file could be done manually ... 
temp_txt = temp_txt.split('#See files sources.txt and stations.txt for more info.')[-1].split('#')[-1]
temp_txt = temp_txt.split('\n')
temp_txt = [line.replace('\r','').strip() for line in temp_txt][1:-1]
temp_header = [col.strip() for col in temp_txt[0].split(',')]
def clean_line(line):
    line = line.split(',')
    line = [col.strip() for col in line]
    # to numeric 
    line = [float(col) for col in line]
    # date
    line[2] = str(int(line[2]))
    return line
temp_list = [clean_line(row) for row in temp_txt[1:]]

df_temp = pd.DataFrame(temp_list, columns=temp_header)
df_temp = df_temp[['DATE', 'TG']]
df_temp.head()

In [ ]:
# temp to celcius 
df_temp['TG'] = df_temp['TG']/10

In [ ]:
# check missing values
df_temp.isnull().sum()

In [ ]:
# timestamp to datetime
df_temp['DATE'] = pd.to_datetime(df_temp['DATE'])

In [ ]:
# reset index
df_temp = df_temp.set_index('DATE')


In [ ]:
df_temp

## Question 5 :

In [ ]:
# merge with df_daily
df_daily = df_daily.join(df_temp, how='inner')
df_daily.head()

In [ ]:

# plot temperature and Global_active_power  between january 1 2007 and april 30 2007 
df_filter = df_daily.loc['2007-01-01':'2007-04-30']

fig, ax = plt.subplots()

df_filter['Global_active_power'].plot(ax=ax, style='b-',label='Global_active_power')
df_filter['TG'].plot(ax=ax, style='r-', secondary_y=True, label ='temperature')

ax.legend([ax.get_lines()[0], ax.right_ax.get_lines()[0]], ['Global_active_power','TG'], loc=0)



- we can see a negative correlation between the temperature  and consuption 
which is expected since we use electricity for heating (a high energy demanding appliance) 

# Exercice 2

### question 6

In [ ]:

def get_left_right_sides(n: int, p: int, lbd: float) -> Tuple[np.ndarray, float, np.ndarray, float]:
    # matrices
    X = np.random.normal(
        loc=0, 
        scale=5**2,
        size=(n, p)
    )

    y = np.random.uniform(
        low=-1, 
        high=1, 
        size=(n, 1)
    )

    id_n = np.eye(n)
    id_p = np.eye(p)

    # operations and inv
    # left
    start_time = time.time()

    transpose_with_id_n = X @ X.T + lbd * id_n
    transpose_id_n_times_y = np.linalg.solve(transpose_with_id_n, y)
    left_side = X.T @ transpose_id_n_times_y

    left_duration = time.time() - start_time

    # right
    start_time = time.time()

    transpose_with_id_p = X.T @ X + lbd * id_p
    transpose_id_p_times_xy = np.linalg.solve(transpose_with_id_p, X.T @ y)  
    right_side = transpose_id_p_times_xy

    right_duration = time.time() - start_time

    return left_side, left_duration, right_side, right_duration

In [ ]:
# 6. a)

# params
n, p = 100, 2000
lbd = 1e-5

l_side, _, r_side, _ = get_left_right_sides(n, p, lbd)

# check equality
all(np.isclose(l_side, r_side, atol=1e-7))

In [ ]:
# 6. b)

# params
n, p = 2000, 100
lbd = 1e-5

l_side, _, r_side, _ = get_left_right_sides(n, p, lbd)

# check equality
all(np.isclose(l_side, r_side, atol=1e-6))

- for this case, the equality holds for an absolute tolerence greater than $10^{-6}$

### Question 7

In [ ]:
# 
def get_duration(n, p):
    _, duration_left, _, duration_right = get_left_right_sides(n, p, lbd)

    return (duration_left, duration_right)


# samples of cases
case_a = [(i, i * 20) for i in range(100, 200+1, 10)]
case_b = [(i * 20, i) for i in range(100, 200+1, 10)]

  
times_case_a = list(
    map(
        lambda tup: get_duration(*tup), 
        case_a
    )
)

times_case_b = list(
    map(
        lambda tup: get_duration(*tup), 
        case_b
    )      
)

In [ ]:
# plot

# init figure
fig = make_subplots(
    rows=1, 
    cols=2,
    subplot_titles=["duration for case n << p", "duration for case n >> p"]
)

color_name = {
    0: {
        "name": "left method",
        "color": '#636EFA'
    },
    1: {
        "name": "right method",
        "color": '#EF553B'
    }
}

fig.add_traces(data=[
    go.Bar(
        x=[str(el) for el in case_a],
        y=[tuple_duration[comp_method] for tuple_duration in times_case_a],
        marker_color=color_name[comp_method]["color"],
        name=color_name[comp_method]["name"],
    )
for comp_method in [0, 1]], 
rows=1, cols=1
)

fig.add_traces(data=[
    go.Bar(
        x=[str(el) for el in case_b],
        y=[tuple_duration[comp_method] for tuple_duration in times_case_b],
        marker_color=color_name[comp_method]["color"],
        name=color_name[comp_method]["name"],
        showlegend=False
    )
for comp_method in [0, 1]], 
rows=1, cols=2
)

# set layout
for i in [1, 2]:
    fig.update_yaxes(
        title_text="time", 
        type="log", 
        row=1, col=i
    )


fig.show()

- It is time efficient to use the left method in the case $n << p$. On the other hand, it turns out that
the right method is the one to opt for in case $n >> p$

- In addition, if we analyse the time complexity of the two operation (without taking on account inv operation)
    - left method: $O(n^2 \ p)$
    - right method: $O(p^2 \ n)$

which shows clearly cases where to choose the method to work with

# Exercice 3

### question 8

In [ ]:
# choosen distributions
DICT_DIST = {
        "uniform": {
            "func": np.random.uniform,
            "mean": 1/2,
            "sigma": np.sqrt(1/12),
            "color": '#636EFA'
        },
        "exponential": {
            "func": np.random.exponential,
            "mean": 1.,
            "sigma": 1.,
            "color": '#EF553B'
        },
        "poisson": {
            "func": np.random.poisson,
            "mean": 1.,
            "sigma": 1.,
            "color": '#00CC96'
        },
    }

TARGET_MEAN = 0
TARGET_VARIANCE = 2


def sample_from(n: int, p: int, dist_name: str) -> np.ndarray:
    # select dist
    dist = DICT_DIST[dist_name]
    # sample from it
    samples_dist = dist["func"](size=(n, p))

    # map it to (mean = 0, var=2)
    return np.sqrt(TARGET_VARIANCE) * ((samples_dist - dist["mean"]) / dist["sigma"] + TARGET_MEAN)

In [ ]:
# numerical check of mean and variance
interval_n = [10 ** i for i in range(1, 6+1)]

diffirence_to_mean = {
    dist_name: [abs(sample_from(n, 1, dist_name).mean() - TARGET_MEAN) for n in interval_n]
    for dist_name in DICT_DIST
}

diffirence_to_variance = {
    dist_name: [abs(sample_from(n, 1, dist_name).var() - TARGET_VARIANCE) for n in interval_n]
    for dist_name in DICT_DIST
}


In [ ]:
# make plot to compare emperical and theorical mean/variance

# init figure
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=[f"abs difference to target mean={TARGET_MEAN}", f"abs difference to target variance={TARGET_VARIANCE}"]
)

# plots to compare mean
fig.add_traces(data=[
    go.Scatter(
        x=interval_n,
        y=diffirence_to_mean[dist_name],
        mode="markers+lines",
        name=dist_name,
        marker_color=DICT_DIST[dist_name]["color"]
    )
for dist_name in DICT_DIST],
rows=1, cols=1)

# plots to compare variance
fig.add_traces(data=[
    go.Scatter(
        x=interval_n,
        y=diffirence_to_variance[dist_name],
        mode="markers+lines",
        name=dist_name,
        marker_color=DICT_DIST[dist_name]["color"],
        showlegend=False
    )
for dist_name in DICT_DIST],
rows=1, cols=2)

# set axis layout
for i in [1, 2]:
    fig.update_xaxes(
        title_text="n (samples)", 
        type="log", 
        row=1, col=i
    )
    
fig.show()

- We notice that the more we increase the number of samples, the more we get close to the theorical mean and variance
- This remark is due to the law of large numbers

### question 9

In [ ]:

# init figure
fig = make_subplots(
    rows=1,
    cols=3,
    subplot_titles=list(DICT_DIST.keys())
)

# params
n = 1000
values_p = [200, 500, 1000, 2000]
arr_colors = ['#636EFA', '#EF553B', '#00CC96', '#AB63FA']

for i, dist_name in enumerate(DICT_DIST):
    for p, color_name in zip(values_p, arr_colors):
        # build and apply svd
        X = sample_from(n, p, dist_name)
        _, singular_vals, _ = np.linalg.svd(X)

        # plot
        fig.add_trace(
            go.Scatter(
                y=singular_vals,
                mode="markers+lines",
                name=f"p={p}",
                marker_color=color_name,
                showlegend=False if dist_name != "uniform" else True
            ),
            row=1, col=i+1
        )

# set axis layout
for i in [1, 2, 3]:
    fig.update_xaxes(
        title_text="singular values",
        row=1, col=i
    )


fig.show()

- We notice that number of singular values is equal to $min(n, p)$. Indeed, the number of singular values is at most
equal to the matrix rank
- We observe that singular values have the same shape for all distributions (when we zoom in, we see that they are slightly different)
- The latter remark can be justified by the fact that all distribution have $mean=0$ and $variance=2$

### question 10

In [ ]:

# init figure
fig = make_subplots(
    rows=1,
    cols=3,
    subplot_titles=list(DICT_DIST.keys())
)

# params
n = 1000
values_p = [200, 500, 1000, 2000]
arr_colors = ['#636EFA', '#EF553B', '#00CC96', '#AB63FA']

for i, dist_name in enumerate(DICT_DIST):
    for p, color_name in zip(values_p, arr_colors):
        # build and apply svd
        X = sample_from(n, p, dist_name)
        eigen_values, _ = np.linalg.eig(X.T @ X / n)

        # plot
        fig.add_trace(
            go.Scatter(
                y=np.real(eigen_values),
                mode="markers+lines",
                name=f"p={p}",
                marker_color=color_name,
                showlegend=False if dist_name != "uniform" else True
            ),
            row=1, col=i+1
        )

# set axis layout
for i in [1, 2, 3]:
    fig.update_xaxes(
        title_text="eigen values",
        row=1, col=i
    )


fig.show()

- Like the previous question, we observe that at the moment $p$ becomes greater than $n=1000$, all eigen values
becomes zeros.

# Exercice 4

### question 11

In [ ]:
# initial implementation
def power_method(X: np.ndarray, max_iterations: int) -> Tuple[np.ndarray, np.ndarray]:
    # X shape
    _, p = X.shape

    # random init of v
    v_k = np.random.rand(p, 1)

    for i in range(max_iterations):
        # update u
        X_times_v = X @ v_k
        u_k = X_times_v / np.linalg.norm(X_times_v)

        # update v
        XT_times_u = X.T @ u_k
        v_k = XT_times_u / np.linalg.norm(XT_times_u)

    return u_k, v_k

In [ ]:
# algo modified
def power_method_modified(X: np.ndarray, max_iterations: int) -> Tuple[List[np.ndarray], List[np.ndarray]]:
    # X shape
    _, p = X.shape

    # random init of v
    arr_v = [np.random.rand(p, 1)]
    arr_u = []

    for i in range(max_iterations):
        # update u
        X_times_v = (X @ arr_v[-1])
        arr_u.append(X_times_v / np.linalg.norm(X_times_v))

        # update v
        XT_times_u = X.T @ (arr_u[-1])
        arr_v.append(XT_times_u / np.linalg.norm(XT_times_u))

    return arr_u, arr_v

In [ ]:
# params
n, p = 5, 10
dist_name = "uniform"

# create matrix
X = sample_from(n, p, dist_name)
U, _, V = np.linalg.svd(X)

# leading singular vectors
u_star = U[:, 0][:, None]
v_star = V.T[:, 0][:, None]

In [ ]:
# plot of u* - u (resp v* - v)

# params
nb_iteration = 200

arr_u, arr_v = power_method_modified(X, max_iterations=nb_iteration)
rg_1000 = list(range(1, nb_iteration + 1))

# init figure
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=["||u* - u||", "||v* - v||"]
)

fig.add_trace(
    go.Scatter(
        x=rg_1000,
        y=[np.linalg.norm(u_star - u) for u in arr_u],
        mode="lines+markers",
        marker_color="#636EFA",
        showlegend=False
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=rg_1000,
        y=[np.linalg.norm(v_star - v) for v in arr_v],
        mode="lines+markers",
        marker_color="#636EFA",
        showlegend=False
    ),
    row=1, col=2
)

# set axis layout
for i in [1, 2]:
    fig.update_xaxes(
        title_text="iteration",
        row=1, col=i
    )

    fig.update_yaxes(
        type="log",
        row=1, col=i
    )

fig.show()

- we can see clearly that as we increase the number of iteration, the distance between $u^*$ and $u$ (resp $v^*$ and $v$)
converge to $0$

### question 14

In [ ]:
# modifed version to approx the max singular value
def power_method_singular_value(X: np.ndarray, max_iterations: int) -> float:
    # X shape
    _, p = X.shape

    # random init of v
    v_k = np.random.rand(p, 1)

    for i in range(max_iterations):
        # update u
        X_times_v = X @ v_k
        u_k = X_times_v / np.linalg.norm(X_times_v)

        # update v
        XT_times_u = X.T @ u_k
        v_k = XT_times_u / np.linalg.norm(XT_times_u)

    return np.sqrt(np.linalg.norm((X.T @ X) @ v_k))

In [ ]:
# params
n, p = 5, 10
dist_name = "uniform"

# create matrix
X = sample_from(n, p, dist_name)
U, sig, V = np.linalg.svd(X)

# singular value using power method
sv = power_method_singular_value(X, max_iterations=1000)


print(np.isclose(sv, sig[0]))

### question 15

In [ ]:
def gen_power_method(
    X: np.ndarray, 
    ith_singular: int, 
    max_iterations: int, 
    v: np.ndarray = None, u: np.ndarray = None
    ) -> Tuple[np.ndarray, np.ndarray, float]:
    
    # condition to break recussion
    if ith_singular == 0:
        sig = np.sqrt(np.linalg.norm((X.T @ X) @ v))
        return v, u, sig
    
    # params
    if (u is not None and
        v is not None):
        sig = np.sqrt(np.linalg.norm((X.T @ X) @ v))
        X = X - sig * u @ v.T

    _, p = X.shape

    # random init of v
    v_k = np.random.rand(p, 1)

    for i in range(max_iterations):
        # update u
        X_times_v = X @ v_k
        u_k = X_times_v / np.linalg.norm(X_times_v)

        # update v
        XT_times_u = X.T @ u_k
        v_k = XT_times_u / np.linalg.norm(XT_times_u)

    # get next sv
    return gen_power_method(X, ith_singular-1, max_iterations, v_k, u_k)

In [ ]:
# params
n, p = 5, 10
dist_name = "uniform"

# create matrix
X = sample_from(n, p, dist_name)
U, sig, V = np.linalg.svd(X)

# 2nd sv using power method
v, u, sv = gen_power_method(X, ith_singular=2, max_iterations=1000)


np.isclose(sv, sig[1])

- The developped function is able to approximate not only the second largest singular value
but also the third, fourth, ... ith singular value

# Exercice 5

### qeustion 16

In [ ]:

# load https://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data-original dataframe 
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data'
header = ['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model year', 'origin', 'car name']
df_auto = pd.read_csv(url, sep='\s+', na_values='?', header=None, names=header)
df_auto


In [ ]:
# check missing values
df_auto.isnull().sum()

In [ ]:
# drop missing values 
df_auto.dropna(inplace=True)

## Question 17 : 

In [ ]:

# add three binary columns : is_US, is_Europe, is_Japan , we will do it in a more ML-ish  way later ...
df_auto['is_US'] = df_auto['origin'] == 1
df_auto['is_Europe'] = df_auto['origin'] == 2
df_auto['is_Japan'] = df_auto['origin'] == 3
df_auto.head()

 ## question 18 : 

In [ ]:
# sample 3 rows from each origin class / we resampled till we found diferent model years ! 
frame_1 = df_auto[df_auto['is_US'] ==True].sample(3)  
frame_2 = df_auto[df_auto['is_Japan'] ==True].sample(3) 
frame_3 =  df_auto[df_auto['is_Europe'] ==True].sample(3)
frames = [frame_1, frame_2, frame_3]

result = pd.concat(frames)


In [ ]:
# get least square estimator 
from sklearn.linear_model import LinearRegression

X_train,y_train = result.loc[:, [ 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model year', 'origin', 'is_US','is_Europe','is_Japan']], result['mpg']

regressor = LinearRegression()
regressor.fit(X_train, y_train)

coefs = regressor.coef_ + regressor.intercept_
coefs = list(regressor.coef_)  + [regressor.intercept_]


In [ ]:
X_test ,y_test = df_auto.loc[:, [ 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model year', 'origin', 'is_US','is_Europe','is_Japan']],df_auto['mpg']
y_pred = regressor.predict(X_test)

In [ ]:
# get the rmse 
from sklearn.metrics import mean_squared_error

rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print('rmse : ', rmse)
print('relative rmse to mean : ', round(rmse/np.mean(y_test),2), '%')
# quite a high error given the mean of our date we cant acceot a 30 % devieation 

In [ ]:
dict(zip( [ 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model year', 'is_US','is_Europe','is_Japan','INTERCEPT'],coefs))

Comments : 

- american cars are more efficient than european  and japaneese cars...  :)
- weight with the least effect on consumption is not realistic ! 

## question 19 : 

In [ ]:

# for pipeline vis purpose 
from sklearn import set_config
set_config(display='diagram') 
# sklearn imports 
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# define feature used for training 
one_hot_encoding_columns = ['origin']
scaling_columns = ['cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model year']
# prepare preprocessor 
# one to onehot encode  the origin column 
# and another one for scaling num vars 
preprocessor = ColumnTransformer([

    ('one-hot-encoder', OneHotEncoder(handle_unknown='ignore'),
     one_hot_encoding_columns),
    ('standard-scaler', StandardScaler(), scaling_columns)
])
model = make_pipeline(
    preprocessor,
    LinearRegression()
)

In [ ]:
# fit the pipeline 
x_columns = scaling_columns+one_hot_encoding_columns
data_train = df_auto.loc[:,x_columns]
target_train = df_auto['mpg']


model.fit(data_train, target_train)


In [ ]:
# get rmse 
target_pred = model.predict(data_train)
rmse_full = np.sqrt(mean_squared_error(target_train, target_pred))
print('rmse : ', rmse_full)
print('relative rmse to mean : ', round(rmse_full/np.mean(target_train),2), '%')

In [ ]:
# get lin_reg coeifs 
coefs_full = list(model.named_steps['linearregression'].coef_) +[ model.named_steps['linearregression'].intercept_]
origine_new_feat_names  = list(model.named_steps['columntransformer'] .named_transformers_['one-hot-encoder'].get_feature_names_out())
feature_names =   origine_new_feat_names+ scaling_columns+['INTERCEPT']
dict(zip(feature_names,coefs_full))

Comments :
-   given the coefs we can say that the weight is the most important feature wich is realisatic since more weight => more consuption 
-  we can see also that there is positive dependency between the year and the consumption whch may cause some problems if we traied the model on futuristic years like 2020 or smtg 

## question 20 :

In [ ]:

observation = [6 ,225 ,100 ,3233 ,15.4, 2017-1900, 1]
observation = np.array(observation).reshape(1,-1)
observation_df = pd.DataFrame(observation, columns=x_columns)
prediction = model.predict(observation_df)[0]
print('prediction : ', prediction)

In [ ]:
prediction , np.mean(target_train)

Comments : (high leverage points)
- the high consumption comes from the year which is  out of sample and already have a high positive coef
 which  drive the prediction up and may be unrealistic . 
- this is equivalent to outliers (whcih we found in target variable) 
- this is found in the regressor variableswe call them **high leverage points** 